In [15]:
import pandas as pd
import time
from tqdm import trange
import json

In [16]:
float_count = 4
int_count = 4

dtypes = {}
for i in range(1, float_count + 1):
    dtypes[f'float_{i}'] = 'float32'
for i in range(1, int_count + 1):
    dtypes[f'int_{i}'] = 'int32'

df = pd.read_csv(f"../../data/synthesis_data.csv", dtype=dtypes)

df["timestamp"] = pd.to_datetime(df["timestamp"])
test_round = 10

print(df.head(), len(df))

            timestamp    float_1    float_2   float_3    float_4  int_1  \
0 2020-01-01 00:00:00  14.830987  15.723351  7.764221   9.139441      8   
1 2020-01-01 00:01:00  15.277334  14.383781  8.050471   9.241592      9   
2 2020-01-01 00:02:00  14.407419  14.133512  7.785035  11.045418      8   
3 2020-01-01 00:03:00  14.416103  14.438385  7.607110   8.510003      9   
4 2020-01-01 00:04:00  15.331453  13.149238  8.170057   8.315617      8   

   int_2  int_3  int_4  
0      7      8      8  
1      7      8      8  
2      7      9      9  
3      7      9      9  
4      7      9      8   3680641


In [17]:
benchmark_results = {
    "metadata": {
        "test_rounds": test_round,
        "data_size": len(df),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
    },
    "where": {},
    "apply": {},
    "sort": {},
    "join": {},
    "groupby": {},
    "interval": {}
}

In [18]:
for i in range(1, int_count + 1):
    start_time = time.time()
    for _ in trange(test_round, desc=f"Where on int_{i}"):
        t = df[df[f"int_{i}"] <= 100]
    benchmark_results["where"][f"int_{i}"] = (time.time() - start_time) * 1000 / test_round

for i in range(1, float_count + 1):
    start_time = time.time()
    for _ in trange(test_round, desc=f"Where on float_{i}"):
        t = df[df[f"float_{i}"] > -55.0]
    benchmark_results["where"][f"float_{i}"] = (time.time() - start_time) * 1000 / test_round

print(benchmark_results["where"])

Where on float_4: 100%|██████████| 10/10 [00:00<00:00, 28.25it/s]

{'int_1': 25.194287300109863, 'int_2': 25.032591819763184, 'int_3': 27.283501625061035, 'int_4': 26.300525665283203, 'float_1': 39.62094783782959, 'float_2': 26.13518238067627, 'float_3': 47.27151393890381, 'float_4': 35.486674308776855}


In [19]:
def double_value(x):
    return x * 2


for i in range(1, float_count + 1):
    start_time = time.time()
    for _ in trange(test_round, desc=f"Apply on float_{i}"):
        t = df[f"float_{i}"].apply(double_value)
    benchmark_results["apply"][f"float_{i}"] = (time.time() - start_time) * 1000 / test_round
    
for i in range(1, int_count + 1):
    start_time = time.time()
    for _ in trange(test_round, desc=f"Apply on int_{i}"):
        t = df[f"int_{i}"].apply(double_value)
    benchmark_results["apply"][f"int_{i}"] = (time.time() - start_time) * 1000 / test_round
    
print(benchmark_results["apply"])

Apply on int_4: 100%|██████████| 10/10 [00:04<00:00,  2.47it/s]

{'float_1': 306.2538146972656, 'float_2': 305.5551052093506, 'float_3': 303.8123607635498, 'float_4': 304.16226387023926, 'int_1': 411.44542694091797, 'int_2': 409.2601776123047, 'int_3': 410.71949005126953, 'int_4': 405.1797389984131}


In [20]:
# Sort 操作
for i in range(1, float_count + 1):
    start_time = time.time()
    for _ in trange(test_round, desc=f"Sort on float_{i}"):
        t = df.sort_values(f"float_{i}")
    benchmark_results["sort"][f"float_{i}"] = (time.time() - start_time) * 1000 / test_round
for i in range(1, int_count + 1):
    start_time = time.time()
    for _ in trange(test_round, desc=f"Sort on int_{i}"):
        t = df.sort_values(f"int_{i}")
    benchmark_results["sort"][f"int_{i}"] = (time.time() - start_time) * 1000 / test_round
print(benchmark_results["sort"])

Sort on int_4: 100%|██████████| 10/10 [00:01<00:00,  8.33it/s]

{'float_1': 171.10111713409424, 'float_2': 173.5069990158081, 'float_3': 174.80392456054688, 'float_4': 175.92267990112305, 'int_1': 114.2702579498291, 'int_2': 117.49212741851807, 'int_3': 117.67563819885254, 'int_4': 120.17946243286133}


In [21]:
# split data by int_1
int_1_span = 7
df_subset_l = df[df["int_1"].between(0, int_1_span)]
df_subset_r = df[df["int_1"].between(int_1_span, int_1_span * 2)]

# print subset len
print(len(df_subset_l))
print(len(df_subset_r))

start_time = time.time()
for i in trange(test_round, desc="Join on int_1"):
    t = df_subset_l.merge(df_subset_r, on="int_2", how="left")
benchmark_results["join"]["int_1"] = (time.time() - start_time) * 1000 / test_round

print(benchmark_results["join"])

6676
190860


Join on int_1:   0%|          | 0/10 [00:00<?, ?it/s]

Join on int_1: 100%|██████████| 10/10 [00:27<00:00,  2.70s/it]

{'int_1': 2702.946400642395}


In [22]:
agg_map = {
    "float_1": ["max", "min", "mean", "sum"],
    "float_2": ["max", "min", "mean", "sum"],
    "float_3": ["max", "min", "mean", "sum"],
    "float_4": ["max", "min", "mean", "sum"],
    "int_2": ["max", "min", "mean", "sum"],
    "int_3": ["max", "min", "mean", "sum"],
    "int_4": ["max", "min", "mean", "sum"],
}

start_time = time.time()
for i in trange(test_round, desc="Groupby on Stkcd"):
    t = df.groupby("int_1").agg(agg_map)
benchmark_results["groupby"]["int_1"] = (time.time() - start_time) * 1000 / test_round
print(benchmark_results["groupby"])

Groupby on Stkcd: 100%|██████████| 10/10 [00:02<00:00,  3.74it/s]

{'int_1': 267.2442674636841}


In [23]:
df = df.set_index("timestamp")
interval_test_round = test_round // int(10)

agg_dict = {
    "float_1": ["max", "min", "mean", "sum", "count"],
    "float_2": ["max", "min", "mean", "sum", "count"],
    "float_3": ["max", "min", "mean", "sum", "count"],
    "float_4": ["max", "min", "mean", "sum", "count"],
    "int_1": ["max", "min", "mean", "sum", "count"],
    "int_3": ["max", "min", "mean", "sum", "count"],
    "int_4": ["max", "min", "mean", "sum", "count"],
}

In [24]:
start_time = time.time()
for i in trange(interval_test_round, desc="Interval 1H"):
    t = df.groupby("int_1").resample("1H").agg(agg_dict)
benchmark_results["interval"]["1H"] = (time.time() - start_time) * 1000 / interval_test_round

start_time = time.time()
for i in trange(interval_test_round, desc="Interval 1D"):
    t = df.groupby("int_1").resample("1D").agg(agg_dict)
benchmark_results["interval"]["1D"] = (time.time() - start_time) * 1000 / interval_test_round

start_time = time.time()
for i in trange(interval_test_round, desc="Interval 1M"):
    t = df.groupby("int_1").resample("30D").agg(agg_dict)
benchmark_results["interval"]["1M"] = (time.time() - start_time) * 1000 / interval_test_round

print(benchmark_results["interval"])

Interval 1H:   0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_345195/3235279237.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  t = df.groupby("int_1").resample("1H").agg(agg_dict)
Interval 1M: 100%|██████████| 1/1 [00:04<00:00,  4.22s/it]

{'1H': 6772.326231002808, '1D': 4235.539674758911, '1M': 4221.1432456970215}


In [25]:
with open("../../results/pandas_syn_benchmark_results.json", "w") as f:
    json.dump(benchmark_results, f, indent=4)